In [57]:
import pandas as pd
import json

In [152]:
#get the output from the information we get
def get_output(res):
    ans = [[] for _ in range(len(res))]
    index = 0
    for key, values in res.items():
        for i in range(len(values)):
            if i == 0:
                ans[index].append((''.join(str(values[i]).split('/'))))
            elif i == 3:
                ans[index].append(ans[index][-1])
            elif i>3:
                tmp = []
                for tms in str(values[i]).split('/'):
                    for tm in tms.split('.'):
                        tmp.append(tm.strip())
                for con in tmp:
                    con_tmps = con.split(' ')
                    for con_tmp in con_tmps:
                        ans[index].append(con_tmp.strip())
        
        number = 13-len(ans[index])
        if number>0:
            for i in range(number):
                if i == 0:
                    ans[index].insert(3, '')
                else:
                    ans[index].insert(i+5, '')
        index+=1
    return ans

In [165]:
#get the information from the dataFrame
def helper(df):
    res = {}
    res[1] = []
    tmp = 1
    for i in range(len(df)):
        block = df.loc[i]
        if (block['Text'].isdigit() and i == 0) or (i > 0 and block['BoundingBox_Left']<df.loc[i-1]['BoundingBox_Left'] and block['Text'].isdigit()):
            res[int(block['Text'])] = []
            tmp = int(block['Text'])
        elif i>0 and block['BoundingBox_Left']>df.loc[i-1]['BoundingBox_Left']:
            res[tmp].append(block['Text'])
    
    new_d = {}
    prev= None
    for k, v in res.items():
        if v:
            new_d[k] = v
            prev=k
        else:
            if prev:
                new_d[prev].append(k)

    return new_d

In [166]:
output = {}
for i in range(5):
    path = './Rule14 Interview Sample Problem/Rule14 Interview Sample Problem/OCR/testimage'+str(i+1)+'.json'
    with open(path, 'r') as f:
        data = json.load(f)
    
    text_list = []
    for block in data['Blocks']:
        if block['BlockType'] == 'LINE':
            bbox = block['Geometry']['BoundingBox']
            polygon = block['Geometry']['Polygon']
            polygon_x = []
            polygon_y = []
            for point in polygon:
                polygon_x.append(point['X'])
                polygon_y.append(point['Y'])
            text_list.append({
                'Text': block['Text'],
                'Confidence': block['Confidence'],
                'BoundingBox_Left': bbox['Left'],
                'BoundingBox_Top': bbox['Top'],
                'BoundingBox_Width': bbox['Width'],
                'BoundingBox_Height': bbox['Height'],
                'Polygon_X': polygon_x,
                'Polygon_Y': polygon_y
            })

    df = pd.DataFrame(text_list)
    res = helper(df)
    da = get_output(res)
    output[str('testimage'+'{}'.format(i+1))] = da
    #print(output)

{'testimage1': [['111020', '111020', '21', '', '95910', '5', '', '', '9', 'in', '1354', '100', 'R'], ['111020', '111020', '21', '', '95861', 'IC', '', '', '', 'A', '3460', 'too', '7'], ['171020', '171020', '20', '2', '1', '9', '5', '9', '2', 'Z', 'TC', 'A', '2664', '100', '1'], ['111020', '111020', '21', '', '95938', 'TO', '', '', '2013', '00', '1', '', ''], ['111020', '111020', '2', '', '95937', 'TC', '', '', '', 'A', '1890', '00', '1'], ['111020', '111020', '2', '', '1', '9595,5', '', '', 'TO', 'A', '4216', 'top', '1']]}
{'testimage1': [['111020', '111020', '21', '', '95910', '5', '', '', '9', 'in', '1354', '100', 'R'], ['111020', '111020', '21', '', '95861', 'IC', '', '', '', 'A', '3460', 'too', '7'], ['171020', '171020', '20', '2', '1', '9', '5', '9', '2', 'Z', 'TC', 'A', '2664', '100', '1'], ['111020', '111020', '21', '', '95938', 'TO', '', '', '2013', '00', '1', '', ''], ['111020', '111020', '2', '', '95937', 'TC', '', '', '', 'A', '1890', '00', '1'], ['111020', '111020', '2', ''

In [167]:
json_str = json.dumps(output)

with open('shuo_wang_results.json', 'w') as f:
    f.write(json_str)